In [ ]:
import pickle
import pandas as pd
import plotly.express as px
import datetime
# from magplots.magFunctions import * # packaged version
# from magFunctions import * # local version
from magplots.magFunctions import *
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(funcName)s - %(message)s', level=logging.DEBUG)

# Event Selection

In [ ]:
with open('themis_event_list_with_coords.pkl', 'rb') as f:
    themis_events = pickle.load(f)
themis_events = themis_events.rename(columns={"Jet Number": "Original Jet Number"})
themis_events['Jet Number'] = np.arange(len(themis_events))+2 # preventing numbers from looping, but making them consistent
themis_events.shape

In [ ]:
# # TESTING - DELETE

# ## Magfig for that day in 2016 with a northern and equatorial jet in the same day:

# # help(magfig)
# events      = [{'datetime':datetime.datetime(2016, 6, 23, 20, 46, 33),'label':'Northern Jet'}]
# events += [{'datetime': datetime.datetime(2016, 6, 23, 23, 21, 54),'label': 'Equatorial Jet'}]
# # magfig(parameter='Bx', start=datetime.datetime(2016, 6, 23, 20, 15), end=datetime.datetime(2016, 6, 23, 21, 46), maglist_a=['upn', 'umq', 'gdh', 'atu', 'skt', 'ghb'], maglist_b=['pg0', 'pg1', 'pg2', 'pg3', 'pg4', 'pg5'], is_displayed=False, is_saved=True, events=events, event_fontdict={'size': 20, 'weight': 'bold'})
# magfig(parameter='Bx', start=datetime.datetime(2016, 6, 23, 20, 15), end=datetime.datetime(2016, 6, 24, 0, 30), maglist_a=['upn', 'umq', 'gdh', 'atu', 'skt', 'ghb'], maglist_b=['pg0', 'pg1', 'pg2', 'pg3', 'pg4', 'pg5'], is_displayed=False, is_saved=True, events=events, event_fontdict={'size': 20, 'weight': 'bold'})
# magfig(parameter='By', start=datetime.datetime(2016, 6, 23, 20, 15), end=datetime.datetime(2016, 6, 24, 0, 30), maglist_a=['upn', 'umq', 'gdh', 'atu', 'skt', 'ghb'], maglist_b=['pg0', 'pg1', 'pg2', 'pg3', 'pg4', 'pg5'], is_displayed=False, is_saved=True, events=events, event_fontdict={'size': 20, 'weight': 'bold'})
# magfig(parameter='Bz', start=datetime.datetime(2016, 6, 23, 20, 15), end=datetime.datetime(2016, 6, 24, 0, 30), maglist_a=['upn', 'umq', 'gdh', 'atu', 'skt', 'ghb'], maglist_b=['pg0', 'pg1', 'pg2', 'pg3', 'pg4', 'pg5'], is_displayed=False, is_saved=True, events=events, event_fontdict={'size': 20, 'weight': 'bold'})



In [ ]:
# for i in [11, 200, 261, 267, 270, 461, 799, 661, 242, 247, 31, 274, 211, 251]:
# for i in [11, 200, 261, 267, 3270, 3461, 799, 3661, 6242, 6247, 1031, 6274, 1211, 4251]:
foo = themis_events[themis_events["Jet Number"].isin([11, 200, 261, 267, 3270, 3461, 799, 3661, 6242, 6247, 1031, 6274, 1211, 4251])]
print(foo [['Original Jet Number','Jet Number' ]])

Add a column named 'Count' that shows the number of events per two-hour period:
<!-- # themis_events['Count'] = themis_events.groupby(themis_events['Start'].dt.date)['Jet Number'].transform('count') -->

In [ ]:
def count_events_within_window_iterative(df):
  """
  This function iterates through each row and calculates the count of events
  within the 2-hour window relative to that row's index time.

  Args:
      df: The pandas DataFrame to be processed.

  Returns:
      A new DataFrame with the added 'Count' column.
  """

  df['Count'] = 0  # Initialize the 'Count' column with zeros
  for index, row in df.iterrows():
    # Extract current row's 'Start' time and window bounds
    start_time = row['Start']
    window_start = start_time - pd.Timedelta(hours=1)
    window_end = start_time + pd.Timedelta(hours=1)

    # Count events within the window (excluding the current row)
    count = (df['Start'] >= window_start) & (df['Start'] < window_end) & (df['Start'] != start_time)
    df.loc[index, 'Count'] = count.sum()  # Assign the count to the current row

  return df

# Apply the function to your DataFrame
themis_events = count_events_within_window_iterative(themis_events)

## Downselect: Data in both hemispheres
We want to select only events with magnetometer coverage from both AALPIP and the DTU stations.

In [ ]:
# themis_events = themis_events[themis_events['Start'] > datetime.datetime(2016, 1, 1)]
themis_events.shape

## Downselect: Check Magnetic Meridian
Consider only the cases where transients occurred within +/-2 hours local time of the 40-degree magnetic meridian. This region is selected because of the density of ground stations available in both hemispheres.

To check this, we'll pull satellite coordinates from https://cdaweb.gsfc.nasa.gov/. Using orbit parameters for spacecraft of interest (e.g., `THA_OR_SSC`), we can find their positions in a cylindrical coordinate system around Earth. We can then check if their longitude in SM coordinates is within 60&deg; of the latitude of our ground stations, since 60&deg; maps to +/- 2 hours out of 24.

In [ ]:
pm = 30   # 30 degrees of margin
themis_events = themis_events.drop(themis_events[(themis_events['SM_LON'] <= 40 - pm) ].index)
themis_events = themis_events.drop(themis_events[(themis_events['SM_LON'] >= 40 + pm) ].index)
themis_events.head()

In [ ]:
themis_events.shape

## Downselect: Filter down to isolated jets: 

In [ ]:
# filter down to isolated jets:
themis_events = themis_events.drop(themis_events[(themis_events['Count'] > 0) ].index)
print(themis_events.shape)
themis_events.head()

## Visualization: Scatterplots
We can plot the downselected events according to various criteria.

In [ ]:
fig = px.scatter(themis_events, x = 'SM_Y', y = 'SM_Z', hover_data = ['Jet Number', 'Start', 'Dynamic Pressure (nPa)'], color = 'DIST_FROM_MAGNETOPAUSE', range_color=(0,1))
fig.update_layout(dragmode='select',
                  newselection=dict(line=dict(color='blue')))
fig.write_html('dist')
fig.show()

In [ ]:
fig = px.scatter(themis_events, x = 'Dynamic Pressure (nPa)', y = 'SM_Z', hover_data = ['Jet Number', 'Start', 'SM_Y', 'Ref Spacecraft'], color = 'DIST_FROM_MAGNETOPAUSE', range_color=(0,1))
fig.update_layout(dragmode='select',
                  newselection=dict(line=dict(color='blue')))
fig.write_html('dist')
fig.show()

In [ ]:
fig = px.scatter(themis_events, x = 'DIST_FROM_MAGNETOPAUSE', y = 'SM_Z', hover_data = ['Jet Number', 'Start', 'SM_Y', 'Ref Spacecraft'], color = 'Dynamic Pressure (nPa)', range_color=(0,1))
fig.update_layout(dragmode='select',
                  newselection=dict(line=dict(color='blue')))
fig.write_html('dist')
fig.show()

In [ ]:
fig = px.scatter(themis_events, x = 'SM_Y', y = 'SM_Z', hover_data = ['Jet Number', 'Start', 'Count', 'Dynamic Pressure (nPa)', 'DIST_FROM_MAGNETOPAUSE', 'DIST_FROM_T95_NS', 'Ref Spacecraft'], 
                 # symbol = 'Ref Spacecraft', 
                 color = 'Dynamic Pressure (nPa)', range_color=(0,10))
fig.update_layout(dragmode='select',
                  newselection=dict(line=dict(color='blue')))
fig.write_html('dyn.html')
fig.show()

In [ ]:
fig = px.scatter(themis_events, x = 'SM_Y', y = 'SM_Z', hover_data = ['Count', 'Jet Number', 'Start', 'Dynamic Pressure (nPa)', 'DIST_FROM_MAGNETOPAUSE', 'DIST_FROM_T95_NS'], color = 'Count', range_color=(0,10))
fig.update_layout(dragmode='select',
                  newselection=dict(line=dict(color='blue')))
fig.write_html('count.html')
fig.show()

# Import selected events with notes:
We manually reviewed the plots according to the following criteria: 

- Overarching Goal 1: intehemispheric comparisons of mesoscale variations related to Pc5 wave activity caused by jets
  - Requirement 1: The jet must be at the same local time as the AALPIP/DTU conjugate network
  - Requirement 2: At least 4 conjugate magnetometers in both hemispheres with mesoscale spacing (AALPIP/DTU conjugate network is the only one of this type) 
  - Requirement 3: Pc5 waves must be clearly observed immediately following the jet
  - Soft Requirement 4: There must be little wave activity happening before the jet to make the case clear that the jet is causing the wave
  - Soft Requirement 5: Immediately before, during, and after the jet the background magnetic activity should be relatively weak so that it’s possible to conduct Fourier analysis without contamination (i.e., no spikes, big dips, etc – this fouls up the Fourier analysis and causes spurious power that might be mistaken for waves)   
 - Overarching Goal 2: Examine at least 2 events with different jet locations but everything else that’s known to affect Pc5 wave amplitudes as similar as possible (local time, day of year, jet dynamic pressure, other solar wind conditions like IMF Bz) to do more of a controlled experiment on the effect of jet position of IH asymmetries
   - Requirement 1: Select at least one jet at positive SM z
   - Requirement 2: Select at least one jet at negative SM z
   - Requirement 3: Have both jets occur at roughly the SM y position (or at least both positive SM y, both negative SM y, etc)
   - Soft Requirement 4 (related to 3): Have both jets occur at roughly the same time of day
   - Soft Requirement 5: Select jets that occur at roughly the same time of year (e.g., both near northern winter, both near spring equinox) to reduce effects related to ionospheric conductance
   - Soft Requirement 6: Select jets with similar dynamic pressures and/or x component of velocity (e.g. within a factor of 2)


In [ ]:
# selected_events = themis_events[themis_events['Jet Number'].isin([31, 461, 242])] #final selection
# selected_events.to_csv('output/selected/selected_events.csv')

selected_events = pd.read_csv('selected_HSJ_events_with_notes.csv')
selected_events["Max"] = pd.to_datetime(selected_events["Max"])
selected_events["Start"] = pd.to_datetime(selected_events["Start"])
selected_events["End"] = pd.to_datetime(selected_events["End"])
selected_events = selected_events.set_index('Max')
selected_events = selected_events[selected_events["Selected"] == True]

### Export table of selected events:

In [ ]:
summary = selected_events.copy()
summary = summary.drop(["Start", "End"], axis = 1)
print(summary.to_markdown())
# print(summary.to_latex())

In [ ]:
# selected_events = themis_events[themis_events["Jet Number"].isin([11, 200, 261, 267, 3270, 3461, 799, 3661, 6242, 6247, 1031, 6274, 1211, 4251])]
selected_events = themis_events[themis_events["Jet Number"].isin([11, 1031, 3461, 3242])]

# Plot Selected Events:

In [ ]:
fig = px.scatter(selected_events, x = 'SM_Y', y = 'SM_Z', hover_data = ['Jet Number', 'Start', 'Dynamic Pressure (nPa)', 'DIST_FROM_MAGNETOPAUSE', 'DIST_FROM_T95_NS', 'Ref Spacecraft'], 
                 # symbol = 'Ref Spacecraft', 
                 color = 'Dynamic Pressure (nPa)', range_color=(0,10))
fig.update_layout(dragmode='select',
                  newselection=dict(line=dict(color='blue')))
fig.write_html('dyn-reduced.html')
fig.show()

## Create time domain, spectrogram plots:

In [ ]:
# themis_events = themis_events.reset_index()
themis_events = selected_events.copy() # comment out to generate ALL THE PLOTS. Comment in if you don't have a day and a half to run this notebook right now.

for i in themis_events.index:
    print('Computing plots for jet #' + str(themis_events['Jet Number'][i]) + ': ')
    events = themis_events[themis_events.index == i].reset_index()[['Max', 'Jet Number']].rename(columns={"Max": "datetime", "Jet Number": "label"}).to_dict('records')
    print(events)
    # start = themis_events['Start'][i].floor('1d')
    # end = start + pd.Timedelta(hours = 24)
    # print(start)
    # print(end)
    for parameter in ['Bx', 'By', 'Bz']:
        try:
            print('Plotting ' + parameter + ' for jet #' + str(themis_events['Jet Number'][i]) + ': ')
            magspect(parameter = parameter, #start = start, end = end,
                start = (themis_events['Start'][i] - pd.Timedelta(hours=2)).to_pydatetime(),
                end = (themis_events['End'][i]+ pd.Timedelta(hours=2)).to_pydatetime(),
                is_logaxis = True,
                # is_logcolor = False,
                is_saved = True, 
                fstem = "selected/Jet " + str(themis_events['Jet Number'][i]) + "_",
                # is_overplotted = False,
                # color = 'red', 
                events = events
            )
            magfig(parameter = parameter, #start = start, end = end,
                # start = themis_events['Start'][1] - pd.Timedelta(hours=2)).to_pydatetime()
                # end = (themis_events['End'][i]+ pd.Timedelta(hours=2)).to_pydatetime(),
                start = (themis_events['Start'][i] - pd.Timedelta(hours=2)).to_pydatetime(),
                end = (themis_events['End'][i]+ pd.Timedelta(hours=2)).to_pydatetime(),
                is_detrended = True,
                is_saved = True, 
                # is_autoscaled = True,
                ylim = [200, 200],
                fstem = "selected/Jet " + str(themis_events['Jet Number'][i]) + "_autoscaled_",
                events = events
            )
        except Exception as e:
            logger.info('Error on the plots for jet #' + str(themis_events['Jet Number'][i]) + ': ')
            logger.info(e)
            print('Continuing to next plot...')
            continue


In [ ]:
            magfig(parameter = parameter, #start = start, end = end,
                # start = themis_events['Start'][1] - pd.Timedelta(hours=2)).to_pydatetime()
                # end = (themis_events['End'][i]+ pd.Timedelta(hours=2)).to_pydatetime(),
                start = (themis_events['Start'][i] - pd.Timedelta(hours=2)).to_pydatetime(),
                end = (themis_events['End'][i]+ pd.Timedelta(hours=2)).to_pydatetime(),
                is_detrended = True,
                is_saved = True, 
                # is_autoscaled = True,
                ylim = [200, 200],
                fstem = "selected/Jet " + str(themis_events['Jet Number'][i]) + "_autoscaled_",
                events = events
            ) #THIS IS A TEST

In [ ]:
    magname = "pg4"
    data = cdas.get_data(
         "sp_phys",
         "THG_L2_MAG_" + str(magname).upper(),
         start,
         end,
        ["thg_mag_" + str(magname)],
    )

In [ ]:
            magfig(parameter = parameter, #start = start, end = end,
                # start = themis_events['Start'][1] - pd.Timedelta(hours=2)).to_pydatetime()
                # end = (themis_events['End'][i]+ pd.Timedelta(hours=2)).to_pydatetime(),
                start = (themis_events['Start'][i] - pd.Timedelta(hours=2)).to_pydatetime(),
                end = (themis_events['End'][i]+ pd.Timedelta(hours=2)).to_pydatetime(),
                is_detrended = True,
                is_saved = True, 
                # is_autoscaled = True,
                ylim = [200, 200],
                fstem = "selected/Jet " + str(themis_events['Jet Number'][i]) + "_autoscaled_",
                events = events
            )

## Generate EIC Quicklook Plots:

In [ ]:
from datetime import datetime

def generate_quicklook_url(date_time):
  """Generates a URL for a quicklook plot based on a datetime object.

  Args:
      date_time: A datetime object representing the desired time.

  Returns:
      A string containing the URL for the quicklook plot.

  Example usage:
        datetime_obj = datetime(year=2016, month=6, day=13, hour=3, minute=33)
        quicklook_url = generate_quicklook_url(datetime_obj)
        print(quicklook_url)
  """
  base_url = "https://vmo.igpp.ucla.edu/data1/SECS/Southernquicklook/"
  year = date_time.year
  month = date_time.month
  day = date_time.day
  hour = str(date_time.hour).zfill(2)  # Convert hour to string before using zfill()
  minute = str(date_time.minute).zfill(2)  # Convert minute to string before using zfill()
  second = str(date_time.second).zfill(2)  # Convert second to string before using zfill()

  # filename = f"SouthEIC{year}{month:02d}{day:02d}_{hour}{minute}{second}.jpg"
  filename = f"SouthEIC{year}{month:02d}{day:02d}_{hour}{minute}00.jpg"
  url = f"{base_url}{year}/{month:02d}/{day:02d}/{filename}"

  return url


for i in selected_events.index:
    datetime_obj = i
    quicklook_url = generate_quicklook_url(datetime_obj)
    print(str(selected_events['Jet Number'][i]) + ": " + quicklook_url)
    # print(quicklook_url)
